In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import os
from keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers

In [ ]:
train_dir=r'C:\Users\itzna\OneDrive\Documents\sih data\dataset\test'
Labels = ['ECZEMA','MELANOMA','BENGIN','ACNE','ATOPIC DERMATITIS','CELLUITIS IMPETIGO','HAIRLOSS','NAIL FUNGUS','PSORIASIS ','RINGWORM']
print ("class : ") 
for i in range(len(Labels)): 
    print (i, end = " ") 
    print (Labels[i])

In [ ]:
print('Number of classes:',len(Labels))


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Define the paths to your train and validation data directories
train_data_dir = r'C:\Users\itzna\OneDrive\Desktop\F_Dataset\train'
validation_data_dir = r'C:\Users\itzna\OneDrive\Desktop\F_Dataset\val'

# Define image size and batch size
img_width, img_height = 160, 160
batch_size = 32

# Create data generators with data augmentation for train and validation sets
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,  # Normalize pixel values
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1.0 / 255.0)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

# Load pre-trained MobileNetV2 model without top layers
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

# Add custom top layers for plant disease detection
num_classes=11

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(num_classes, activation='softmax')(x)  # 'num_classes' is the number of disease classes

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the pre-trained layers in the base model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
epochs = 80# Adjust the number of epochs as needed
history =model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=len(validation_generator)
)

# Save the trained model for future use
model.save('disease_detection_model50.h5')



In [ ]:
model.summary()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Assuming you have a 'history' object obtained after training a model
# Replace this with the actual 'history' object you have from your model training
history = {
    'accuracy': [0.6547, 0.66573, 0.6556, 0.6571],  # Replace with your actual training accuracy values
    'val_accuracy': [0.6319, 0.6335, 0.6254, 0.6453],  # Replace with your actual validation accuracy values
    'loss': [0.8330, 0.8236, 0.8202, 0.8144],  # Replace with your actual training loss values
    'val_loss': [0.9359, 0.9150, 0.9252, 0.8283]  # Replace with your actual validation loss values
}

epochs = len(history['accuracy'])
epochs_range = range(1, epochs + 1)

plt.figure(figsize=(10, 4))

# Plotting Accuracy
plt.subplot(1, 2, 1)
plt.plot(epochs_range, history['accuracy'], label='Training Accuracy')
plt.plot(epochs_range, history['val_accuracy'], label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')

# Plotting Loss
plt.subplot(1, 2, 2)
plt.plot(epochs_range, history['loss'], label='Training Loss')
plt.plot(epochs_range, history['val_loss'], label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')

plt.tight_layout()
plt.show()



In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the paths to your train and validation data directories
train_data_dir = r'C:\Users\itzna\OneDrive\Desktop\F_Dataset\train'
validation_data_dir = r'C:\Users\itzna\OneDrive\Desktop\F_Dataset\val'

# Define image size and batch size
img_width, img_height = 160, 160
batch_size = 32

# Create data generators with data augmentation for train and validation sets
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,  # Normalize pixel values
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1.0 / 255.0)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)
# Define the CNN model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),  # Adding dropout for regularization
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(len(train_generator.class_indices), activation='softmax')  # Output layer
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])



# Predict the classes using the model
predictions = model.predict(validation_generator)
y_pred = np.argmax(predictions, axis=1)  # Predicted labels

# True labels
y_true = validation_generator.classes

# Calculate confusion matrix
cm = confusion_matrix(y_true, y_pred)
# Create a DataFrame for better visualization
class_names = list(validation_generator.class_indices.keys())
df_cm = pd.DataFrame(cm, index=class_names, columns=class_names)

# Plot the confusion matrix
plt.figure(figsize=(10, 7))
sns.heatmap(df_cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

In [ ]:

from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import datasets

# Load your skin disease dataset (replace 'data' and 'target' with your actual data)
data = datasets.load_digits()
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.2, random_state=42)

# Create an SVM classifier
svm_classifier = svm.SVC(kernel='linear')

# Train the SVM classifier
svm_classifier.fit(X_train, y_train)

# Make predictions on the test set
svm_predictions = svm_classifier.predict(X_test)

# Evaluate accuracy
svm_accuracy = accuracy_score(y_test, svm_predictions)
print(f"SVM Accuracy: {svm_accuracy}")

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns


# Create confusion matrix
svm_conf_matrix = confusion_matrix(y_test, svm_predictions)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(svm_conf_matrix, annot=True, fmt="d", cmap="Blues", cbar=False)
plt.title('SVM Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import datasets

# Load your skin disease dataset (replace 'data' and 'target' with your actual data)
data = datasets.load_digits()
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.2, random_state=42)

# Create a Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the Random Forest classifier
rf_classifier.fit(X_train, y_train)

# Make predictions on the test set
rf_predictions = rf_classifier.predict(X_test)

# Evaluate accuracy
rf_accuracy = accuracy_score(y_test, rf_predictions)
print(f"Random Forest Accuracy: {rf_accuracy}")


In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Create confusion matrix
rf_conf_matrix = confusion_matrix(y_test, rf_predictions)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(rf_conf_matrix, annot=True, fmt="d", cmap="Greens", cbar=False)
plt.title('Random Forest Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

# Load the saved model
saved_model_path = r"C:\Users\itzna\prj\disease_detection_model50.h5"
model = keras.models.load_model(saved_model_path)

# Define the path to the new image you want to test
test_image_path = r"C:\Users\itzna\OneDrive\Documents\sih data\dataset\data\Benign Keratosis\ISIC_0025743.jpg"

# Load and preprocess the test image
img = keras.preprocessing.image.load_img(test_image_path, target_size=(160, 160))
img_array = keras.preprocessing.image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0  # Normalize the image

# Make predictions using the loaded model
predictions = model.predict(img_array)

# Assuming you have a list of class labels, you can map the class indices to class labels
class_labels = ['ACNE','ATOPIC DERMATITIS','BENGIN KERATOSIS','CELLULITIS','ECZEMA','HAIRLOSS/ALOPECIA','MELANOMA','NAIL FUNGUS/PARONYCHIA','PSORIASIS ','TINEA/RINGWORM']  # Replace with your class labels
predicted_class_label = class_labels[np.argmax(predictions)]

# Define precautions for each disease
precautions = {
    'ACNE': ['Keep your skin clean', 'Shampoo your hair regularly', 'Stop trying new acne treatments too often as they can irritate your skin and cause breakouts','Stay out of the sun and tannig beds'],
    'ATOPIC DERMATITIS': ['Avoid scratching the rash or skin', 'Prevent food items like wheat, soy, sesame seeds, broccoli, peanuts etc', 'Preventing dust mites, pets pollens etc which cause allergy'],
    'BENGIN KERATOSIS': ['There is no way to completely prevent the development of seborrheic keratoses'],
    'CELLULITIS': ['Aoid injuring your skin', 'Avoid scratching and wearing 100% cotton clothes', 'Stress management','To eat more anti-inflammatory foods like leafy vegetables, turmeric, beans and lentils'],
    'ECZEMA': ['Use gentle soaps and detergents', 'Avoid triggers like allergens', 'Use mild fragrance-free skincare products'],
    'HAIRLOSS/ALOPECIA': ['Drink plenty of water', 'Use mild shampoo', 'Learn how to manage stress','Eat balanced diet and have good sleep'],
    'MELANOMA': ['Limit or avoid direct exposure to the sun between 10:00 AM and 4:00 PM', 'Wear sun-protective clothing', 'Use a broad-spectrum sunscreen throughout the year that protects against both UVA and UVB radiation','Do not use sun lamps, tanning devices, or tanning salons'],
    'NAIL FUNGUS/PARNOYCHIA': ['Avoiding biting or chewing the nails', 'Avoiding submerging the hands in water for long periods', 'Avoiding contact with irritants','Keeping the hands and nails clean'],
    'PSORIASIS': ['Use Moisturizing Creams', 'Take Care of Your Skin', 'Avoid Dry, Cold Weather','Use a Humidifier to keep your skin moist'],
    'TINEA/RINGWORM': ['Wear shoes that allow air to circulate freely around your feet', 'Dont walk barefoot in areas like locker rooms or public showers', 'Avoid using very hot water','Always use a fresh dry towel to wipe off water from the body']
}

# Display the input image
plt.figure(figsize=(6, 6))
plt.imshow(img)
plt.title(f'Predicted Disease: {predicted_class_label}')
plt.axis('off')
plt.show()

# Print the predicted class label
print(f'The model predicts that the image belongs closely to this class: {predicted_class_label}')

# Print precautions for the detected disease
if predicted_class_label in precautions:
    print('Precautions:')
    for precaution in precautions[predicted_class_label]:
        print(f'- {precaution}')
else:
    print('No precautions available for this disease.')
